In [87]:
# Use the data folder provided in Tutorial 02 - Intro to Pandas.
DATA_FOLDER = 'Data/' 
EBOLA_FOLDER = DATA_FOLDER + 'ebola/'

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [79]:
import glob
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

def import_from_forder(folder_path):
    allFiles = glob.glob(folder_path + '/*.csv')
    frame = pd.DataFrame()
    list_ = []

    for file_ in allFiles:
        df = pd.read_csv(file_, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    return frame

# import the 3 subfolders
folders = [x[0] for x in os.walk(EBOLA_FOLDER)][1:]

# dataframes from 3 different folders
df1,df2,df3 = [import_from_forder(x) for x in folders]

# get same notation for the 3 dataframes and replace '-' by '0'
df1 = df1.rename(columns = {'Totals':'Guinea'}).replace(to_replace="-", value=0)
df2 = df2.rename(columns = {'Variable':'Description', 'National':'Liberia'}).replace(to_replace="-", value=0)
df3 = df3.rename(columns = {'date':'Date', 'variable':'Description', 'National':'Sierra Leon'}).replace(to_replace="-", value=0)

In [80]:
def filter_by_terms(df, terms, match=True):
    if match:
        return df.loc[df.Description.str.match('|'.join(terms))]
    else:
        return df.loc[df.Description.str.contains('|'.join(terms))]

# take the rows containing deaths and cases terms and changing description to 'death' and 'case'
# returns 2 dataframes
def separate_death_case(df, death_terms, case_terms):
    return1 = filter_by_terms(df, death_terms)
    return1.Description = return1.Description.map(lambda x : 'death')
    return2 = filter_by_terms(df, case_terms)
    return2.Description = return2.Description.map(lambda x : 'case')
    return return1,return2

# specific terms we chose to be meaningful
# we only consider confirmed new cases and confirmed new deaths
df1_death_terms = ['New deaths registered', 'New deaths registered today']
df1_case_terms = ['New cases of confirmed']
df2_death_terms = ['Newly reported deaths']
df2_case_terms = ['New case/s \(confirmed\)']
df3_death_terms = ['etc_new_deaths']
df3_case_terms = ['new_confirmed']
    
df1_death,df1_case = separate_death_case(df1, df1_death_terms, df1_case_terms)
df2_death,df2_case = separate_death_case(df2, df2_death_terms, df2_case_terms)
df3_death,df3_case = separate_death_case(df3, df3_death_terms, df3_case_terms)

# fill NaN in totals with the sum of the row
def to_numeric(df):
    return df.apply(pd.to_numeric, errors="ignore")

df1_death.Guinea = to_numeric(df1_death).Guinea.fillna(df1_death.drop(['Guinea']).sum(axis=1))
df1_case.Guinea = to_numeric(df1_case).Guinea.fillna(df1_case.drop(['Guinea']).sum(axis=1))

df2_death.Liberia = to_numeric(df2_death).Liberia.fillna(df2_death.drop(['Liberia']).sum(axis=1))
df2_case.Liberia = to_numeric(df2_case).Liberia.fillna(df2_case.drop(['Liberia']).sum(axis=1))

df3_death["Sierra Leon"] = to_numeric(df3_death)["Sierra Leon"].fillna(df3_death.drop(["Sierra Leon"]).sum(axis=1))
df3_case["Sierra Leon"] = to_numeric(df3_case)["Sierra Leon"].fillna(df3_case.drop(["Sierra Leon"]).sum(axis=1))

# list of all dataframes containing death and case terms
list_df = [df1_death, df1_case, df2_death, df2_case, df3_death, df3_case]

In [84]:
# concatenate the three dataframes and reorder columns
df = pd.concat(list_df)
df = df[['Date', 'Description', 'Guinea', 'Liberia', 'Sierra Leon']]

# cast right types
df.apply(pd.to_numeric, errors='ignore')
df.Date = pd.to_datetime(df.Date)

# groupe by date and description
# pd.set_option('precision',2)
# df.groupby([df.Description, df.Date.dt.to_period("M")]).mean().fillna('')

In [85]:
cases = df[df.Description == "case"]
cases.groupby([cases.Date.dt.to_period("M")]).mean().fillna('')

,Guinea,Liberia,Sierra Leon
Date,,,
2014-06,,2.14,
2014-07,,1.82,
2014-08,6.9,5.44,19
2014-09,6.7,6.17,34
2014-10,4,1.36,57
2014-11,,2.60,60
2014-12,,1928.33,33


In [86]:
deaths = df[df.Description == "death"]
deaths.groupby([deaths.Date.dt.to_period("M")]).mean().fillna('')

,Guinea,Liberia,Sierra Leon
Date,,,
2014-06,,2.00,
2014-07,,4.27,
2014-08,1.6,23.22,0
2014-09,1.8,36.04,0.97
2014-10,7.5,28.04,3.7
2014-11,,13.47,0.57
2014-12,,0.00,2.2
